In [3]:
import seaborn as sns
import pandas as pd
import psycopg2

In [6]:
conn = psycopg2.connect(host="localhost",database="tratamento_efetivo", user="postgres", password="1234")

# Relatório 1

#### Relatório para informar os tratamentos em andamento

In [25]:
relatorio1 = pd.read_sql_query("""
select pac.nome_paciente,indisp.indisposicao as 
"Tratamento", med.nome_medico, trat.ini_trat,trat.fim_trat,hosp.hospital from tratamento trat
inner join paciente pac on (pac.cod_paciente = trat.cod_paciente) inner join medico med on (trat.cod_medico = med.cod_medico)
inner join indisposicao indisp on (trat.cod_indisposicao = indisp.cod_indisposicao) 
inner join hospital hosp on (trat.cod_hospital = hosp.cod_hospital) where fim_trat is null
                            """,conn)

In [26]:
relatorio1

,nome_paciente,Tratamento,nome_medico,ini_trat,fim_trat,hospital
0,João Pedro,Insônia,Alfredo Guerra Tutsk,2017-02-18,None,Hospital Regional Dantas Biao
1,Miguel Pereira,Pressão alta,Priscila Catrina,2016-08-10,None,Hospital Santa Cruz
2,Thiago Correia Neves,Depressão,Sabrina Fantin,2017-06-17,None,Hospital Israelita Albert Einstein


# Relatório 2

#### Relatório que retorna quais hospitais os tratamentos foram finalizados


In [27]:
relatorio2 = pd.read_sql_query("""
select hosp.hospital,indisp.indisposicao 
as "Tratamento", trat.ini_trat,trat.fim_trat from tratamento trat
inner join hospital hosp on (trat.cod_hospital = hosp.cod_hospital)
inner join indisposicao indisp on (trat.cod_indisposicao = indisp.cod_indisposicao)
where fim_trat is not null
                            """,conn)

In [63]:
relatorio2

,hospital,Tratamento,ini_trat,fim_trat
0,Hospital Vitória Apart,Gripe,2017-11-15,2017-11-20
1,Hospital São Camilo,Asma,2017-03-02,2017-04-01
2,Hospital São Domingos,Pneumonia,2018-12-14,2018-12-28
3,Clínica Viver Bem,Labirintite,2019-02-16,2019-03-02
4,Hospital São Lucas Copacabana,Gastroenterite,2018-09-05,2018-09-10
5,Hospital Paineiras,Sinusite,2017-09-02,2017-09-17
6,Hospital Santa Rosália,Conjuntivite,2015-08-16,2015-08-23


# Relatório 3

#### Relatório que informa quantos tratamentos foram finalizados no hospital, desde que tenha tratamentos registrado no hospital


In [83]:
relatorio3 = pd.read_sql_query("""
select hosp.hospital as hospital,count(trat.cod_hospital) as quantidade_de_tratamentos_realizado_pelo_hospital from tratamento trat
inner join hospital hosp on (trat.cod_hospital = hosp.cod_hospital) where fim_trat is not null group by hosp.cod_hospital;
                            """,conn)

In [84]:
relatorio3

,hospital,quantidade_de_tratamentos_realizado_pelo_hospital
0,Hospital São Lucas Copacabana,1
1,Hospital Vitória Apart,1
2,Clínica Viver Bem,1
3,Hospital São Domingos,1
4,Hospital Santa Rosália,1
5,Hospital São Camilo,1
6,Hospital Paineiras,1


In [87]:
import matplotlib.pyplot as plt; plt.rcdefaults()
import numpy as np
import matplotlib.pyplot as plt
 
objects = tuple(df.hospital)
y_pos = np.arange(len(objects))
performance = list(df.quantidade_de_tratamentos_realizado_pelo_hospital)
 
plt.bar(y_pos, performance, align='center', alpha=0.5)
plt.xticks(y_pos, objects)
plt.ylabel('Hospital')
plt.title('Número de tratamentos finalizados')
 
plt.show()

AttributeError: 'DataFrame' object has no attribute 'quantidade_de_tratamentos_realizado_pelo_hospital'

# Relatório 4

#### Relatório que informa quanto durou cada tratamento finalizado

In [78]:
relatorio4 = pd.read_sql_query("""
select pac.nome_paciente,indisp.indisposicao as 
"Tratamento", trat.ini_trat,trat.fim_trat, age(fim_trat,ini_trat) as "Duração do tratamento" 
from tratamento trat inner join paciente pac on (pac.cod_paciente = trat.cod_paciente)
inner join indisposicao indisp on (trat.cod_indisposicao = indisp.cod_indisposicao) 
where fim_trat is not null
                            """,conn)

In [79]:
relatorio4

,nome_paciente,Tratamento,ini_trat,fim_trat,Duração do tratamento
0,José Alves,Gripe,2017-11-15,2017-11-20,5 days
1,Maria Souza,Asma,2017-03-02,2017-04-01,30 days
2,Pedro Santos,Pneumonia,2018-12-14,2018-12-28,14 days
3,Ana Kruger Santos,Labirintite,2019-02-16,2019-03-02,14 days
4,Davi Santos,Gastroenterite,2018-09-05,2018-09-10,5 days
5,Clarice Pereira Silva,Sinusite,2017-09-02,2017-09-17,15 days
6,Joana Medeiros,Conjuntivite,2015-08-16,2015-08-23,7 days


# Relatório 5

#### Relatório contendo quantos medicamentos o paciente está usando em cada tratamento

In [88]:
relatorio5 = pd.read_sql_query("""
select pac.nome_paciente,indisp.indisposicao as tratamento, count(med.cod_tratamento) as qtde_medicamento from tratamento trat
inner join paciente pac on (trat.cod_paciente = pac.cod_paciente) 
inner join indisposicao indisp on (trat.cod_indisposicao = indisp.cod_indisposicao)
inner join medicamento med on (trat.cod_tratamento = med.cod_tratamento)
group by pac.cod_paciente,trat.cod_paciente,med.cod_tratamento,indisp.cod_indisposicao;
                            """,conn)

In [89]:
relatorio5

,nome_paciente,tratamento,qtde_medicamento
0,Joana Medeiros,Conjuntivite,1
1,Davi Santos,Gastroenterite,1
2,Pedro Santos,Pneumonia,1
3,Maria Souza,Asma,1
4,João Pedro,Insônia,1
5,Ana Kruger Santos,Labirintite,2
6,Thiago Correia Neves,Depressão,1
7,José Alves,Gripe,1
8,Clarice Pereira Silva,Sinusite,1
9,Miguel Pereira,Pressão alta,1
